In [110]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import numpy as np
import sys
Stopwords = set(stopwords.words('indonesian'))
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [111]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq

def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex,'',text)
    return text_returned

In [112]:

all_words = []
dict_global = {}
file_folder = 'export/*'
idx = 1
files_with_index = {}
for file in glob.glob(file_folder):
    print(file)
    fname = file
    file = open(file , "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'),'',text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    words = [stemmer.stem(word) for word in words]
    dict_global.update(finding_all_unique_words_and_freq(words))
    files_with_index[idx] = os.path.basename(fname)
    idx = idx + 1
unique_words_all = set(dict_global.keys())


export\edukasi1.txt
export\edukasi10.txt
export\edukasi2.txt
export\edukasi3.txt
export\edukasi4.txt
export\edukasi5.txt
export\edukasi6.txt
export\edukasi7.txt
export\edukasi8.txt
export\edukasi9.txt
export\food1.txt
export\food10.txt
export\food2.txt
export\food3.txt
export\food4.txt
export\food5.txt
export\food6.txt
export\food7.txt
export\food8.txt
export\food9.txt
export\money1.txt
export\money10.txt
export\money2.txt
export\money3.txt
export\money4.txt
export\money5.txt
export\money6.txt
export\money7.txt
export\money8.txt
export\money9.txt
export\sains1.txt
export\sains10.txt
export\sains2.txt
export\sains3.txt
export\sains4.txt
export\sains5.txt
export\sains6.txt
export\sains7.txt
export\sains8.txt
export\sains9.txt
export\tekno1.txt
export\tekno10.txt
export\tekno2.txt
export\tekno3.txt
export\tekno4.txt
export\tekno5.txt
export\tekno6.txt
export\tekno7.txt
export\tekno8.txt
export\tekno9.txt


In [113]:
class Node:
    def __init__(self ,docId, freq = 0):
        self.freq = freq
        self.tfidf = 0
        self.doc = docId
        self.nextval = None
class SlinkedList:
    def __init__(self ,head = None, nDoc=0):
        self.head = head
        self.ndoc = nDoc
        self.idf = 0

In [114]:
linked_list_data = {}
for word in unique_words_all:
    linked_list_data[word] = SlinkedList()
    linked_list_data[word].head = Node(1,Node)
word_freq_in_doc = {}
idx = 0
for file in glob.glob(file_folder):
    file = open(file, "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'),'',text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    words = [stemmer.stem(word) for word in words]
    word_freq_in_doc = finding_all_unique_words_and_freq(words)
    for word in word_freq_in_doc.keys():
        linked_list = linked_list_data[word].head
        linked_list_data[word].ndoc=linked_list_data[word].ndoc+1
        while linked_list.nextval is not None:
            linked_list = linked_list.nextval
        linked_list.nextval = Node(idx ,word_freq_in_doc[word])
    idx = idx + 1

In [115]:
import math
import operator
nTot=len(files_with_index)
for word in unique_words_all:
    linkedlist = linked_list_data[word].head
    df=linked_list_data[word].ndoc
    idf=0
    idf = math.log2(nTot/df)+1
    linked_list_data[word].idf=idf
    while linkedlist.nextval is not None:
        linkedlist = linkedlist.nextval
        linkedlist.tfidf = idf * linkedlist.freq

In [116]:
dict_words=list(unique_words_all)
total_files = len(files_with_index)
total_vocab = len(dict_words)
vecD = np.zeros((total_files,total_vocab))
for i in range(len(dict_words)):
    linkedlist = linked_list_data[dict_words[i]].head
    while linkedlist.nextval is not None:
        linkedlist = linkedlist.nextval
        vecD[linkedlist.doc - 1][i] = linkedlist.tfidf

In [117]:
query = input('Enter your query:')
dict_query = {}
vecQ = np.zeros(total_vocab)
wordsq = word_tokenize(query)
# wordsq = [word for word in wordsq if len(wordsq)>1]
wordsq = [word.lower() for word in wordsq]
wordsq = [word for word in wordsq if word not in Stopwords]
wordsq = [stemmer.stem(word) for word in wordsq]
dict_query.update(finding_all_unique_words_and_freq(wordsq))
for word in dict_query.keys():
    print(word)
    vecQ[dict_words.index(word)] = linked_list_data[word].idf * dict_query[word]


baik


In [118]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [119]:
d_cosines = []
for d in vecD:
    d_cosines.append(cosine_sim(vecQ,d))
    # print(d_cosines)
out=np.array(d_cosines).argsort()[-total_files:][::-1]
print(out)

[25  2  1  4 43 24  0 45 19  3 47 35 34  7 42 12 17 16 15 14 13  8 11 10
  9  6  5 46 18 21 20 41 40 39 38 37 36 33 32 31 30 29 28 27 26 44 48 23
 22 49]
